# Langchain (v0.3) 의 주요 Components
https://python.langchain.com/docs/how_to/#components

1. Chat models
- Chat model 은 '메시지'를 입력받아 출력하는 형태의 언어 모델. 

1. Messages

1. Prompt templates

1. Example selectors

1. LLMs

1. Output parsers

1. Document loaders

1. Text splitters

1. Embedding models

1. Vector stores

1. Retrievers


1. Chains
https://python.langchain.com/docs/versions/migrating_chains/


1. Agents

1. Memory

1. Callbakcs


# prompts
https://python.langchain.com/api_reference/core/prompts.html#module-langchain_core.prompts

In [1]:
"""
prompts class 계층도

BasePromptTemplate --> PipelinePromptTemplate
                       StringPromptTemplate --> PromptTemplate
                                                FewShotPromptTemplate
                                                FewShotPromptWithTemplates
                       BaseChatPromptTemplate --> AutoGPTPrompt
                                                  ChatPromptTemplate --> AgentScratchPadChatPromptTemplate



BaseMessagePromptTemplate --> MessagesPlaceholder
                              BaseStringMessagePromptTemplate --> ChatMessagePromptTemplate
                                                                  HumanMessagePromptTemplate
                                                                  AIMessagePromptTemplate
                                                                  SystemMessagePromptTemplate
"""
None

# FewShotPromptTemplate

## 기본 import

In [2]:
# v0.3
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [3]:
chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler()
    ],
)

In [4]:
# PromptTemplate 사용 방법
# 방법1
t = PromptTemplate.from_template("What is the capital of {country}")
t.format(country='France')

'What is the capital of France'

In [5]:
# 방법2
t2 = PromptTemplate(
    template="What is the capital of {country}",
    input_variables=['country']
)
t2.format(country='France')

'What is the capital of France'

## 모델에 예제(example) 주기 

In [6]:
# 모델에게 '어떻게 대답해야 하는 지에 대한 예제(example)'를 AI 모델에게 주는 것이
# prompt 를 사용해서 '어떻게 대답해야 하는지 알려주는 것'보다 훨씬 좋다

# FewShotPromptTemplate 이 하는 일이 바로 그거다!
# - 이를 통해 예제(샘플)를 형식화(포맷) 할수 있다.
# - 이런 예제들을 데이터베이스등에 저장시켜놓고 활용할수도 있다

In [7]:
# v0.3
from langchain_core.prompts.few_shot import FewShotPromptTemplate

In [8]:
examples = [
  {
    "question": "What do you know about France?",

    # ↓ 원하는 형식의 답변 이다..
    "answer": """
      Here is what I know:
      Capital: Paris
      Language: French
      Food: Wine and Cheese
      Currency: Euro
      """,
  },
  {
    "question": "What do you know about Italy?",
    "answer": """
      I know this:
      Capital: Rome
      Language: Italian
      Food: Pizza and Pasta
      Currency: Euro
      """,
  },
  {
    "question": "What do you know about Greece?",
    "answer": """
      I know this:
      Capital: Athens
      Language: Greek
      Food: Souvlaki and Feta Cheese
      Currency: Euro
      """,
  },    
]

In [9]:
# example 없이 전달하면?
chat.invoke("What do you know about France?")

France is a country located in Western Europe. It is known for its rich history, culture, and cuisine. The capital city is Paris, which is famous for landmarks such as the Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral.

France is the largest country in the European Union by land area and the third-largest in Europe overall. It has a population of around 67 million people. French is the official language, and the currency is the Euro.

France is a popular tourist destination, attracting millions of visitors each year to its beautiful cities, countryside, and coastline. The country is also known for its wine and cheese production, fashion industry, and art and literature.

France has a long history of political and cultural influence, including the French Revolution, the Enlightenment, and the development of democracy and human rights. It is a founding member of the United Nations, NATO, and the European Union.

AIMessage(content='France is a country located in Western Europe. It is known for its rich history, culture, and cuisine. The capital city is Paris, which is famous for landmarks such as the Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral.\n\nFrance is the largest country in the European Union by land area and the third-largest in Europe overall. It has a population of around 67 million people. French is the official language, and the currency is the Euro.\n\nFrance is a popular tourist destination, attracting millions of visitors each year to its beautiful cities, countryside, and coastline. The country is also known for its wine and cheese production, fashion industry, and art and literature.\n\nFrance has a long history of political and cultural influence, including the French Revolution, the Enlightenment, and the development of democracy and human rights. It is a founding member of the United Nations, NATO, and the European Union.', additional_kwargs={}, response_metadata={'

In [10]:
example_template = """
    Human: {question}
    AI: {answer}
"""

In [11]:
example_prompt = PromptTemplate.from_template(example_template)
example_prompt

PromptTemplate(input_variables=['answer', 'question'], input_types={}, partial_variables={}, template='\n    Human: {question}\n    AI: {answer}\n')

In [12]:
prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples = examples,

    suffix = "Human: What do you know abount {country}?",
    input_variables=['country']
)

prompt

FewShotPromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, examples=[{'question': 'What do you know about France?', 'answer': '\n      Here is what I know:\n      Capital: Paris\n      Language: French\n      Food: Wine and Cheese\n      Currency: Euro\n      '}, {'question': 'What do you know about Italy?', 'answer': '\n      I know this:\n      Capital: Rome\n      Language: Italian\n      Food: Pizza and Pasta\n      Currency: Euro\n      '}, {'question': 'What do you know about Greece?', 'answer': '\n      I know this:\n      Capital: Athens\n      Language: Greek\n      Food: Souvlaki and Feta Cheese\n      Currency: Euro\n      '}], example_prompt=PromptTemplate(input_variables=['answer', 'question'], input_types={}, partial_variables={}, template='\n    Human: {question}\n    AI: {answer}\n'), suffix='Human: What do you know abount {country}?')

In [13]:
print(prompt.format(country = 'Germany'))


    Human: What do you know about France?
    AI: 
      Here is what I know:
      Capital: Paris
      Language: French
      Food: Wine and Cheese
      Currency: Euro
      



    Human: What do you know about Italy?
    AI: 
      I know this:
      Capital: Rome
      Language: Italian
      Food: Pizza and Pasta
      Currency: Euro
      



    Human: What do you know about Greece?
    AI: 
      I know this:
      Capital: Athens
      Language: Greek
      Food: Souvlaki and Feta Cheese
      Currency: Euro
      


Human: What do you know abount Germany?


In [14]:
chain = prompt | chat

In [15]:
chain.invoke({
    'country': "Germany"
})

AI: 
      Here is what I know:
      Capital: Berlin
      Language: German
      Food: Bratwurst and Sauerkraut
      Currency: Euro

AIMessage(content='AI: \n      Here is what I know:\n      Capital: Berlin\n      Language: German\n      Food: Bratwurst and Sauerkraut\n      Currency: Euro', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-3.5-turbo-0125'}, id='run--1692c0a4-eb2a-48dc-9bc0-f0e62b1bf047-0')

In [16]:
chain.invoke({
    'country': "Turkey"
})

AI: 
      Here is what I know:
      Capital: Ankara
      Language: Turkish
      Food: Kebab and Baklava
      Currency: Turkish Lira

AIMessage(content='AI: \n      Here is what I know:\n      Capital: Ankara\n      Language: Turkish\n      Food: Kebab and Baklava\n      Currency: Turkish Lira', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-3.5-turbo-0125'}, id='run--8040fcc2-f7f6-4d18-b588-e5ae7008f0b4-0')

# FewShotChatMessagePromptTemplate

In [17]:
# v0.3
from langchain_core.prompts.few_shot import FewShotChatMessagePromptTemplate
# https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.few_shot.FewShotChatMessagePromptTemplate.html

# v0.3
from langchain_core.prompts.chat import ChatPromptTemplate
# https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html

In [18]:
examples = [
  {
    "country": "France",

    # ↓ 원하는 형식의 답변 이다..
    "answer": """
      Here is what I know:
      Capital: Paris
      Language: French
      Food: Wine and Cheese
      Currency: Euro
      """,
  },
  {
    "country": "Italy",
    "answer": """
      I know this:
      Capital: Rome
      Language: Italian
      Food: Pizza and Pasta
      Currency: Euro
      """,
  },
  {
    "country": "Greece",
    "answer": """
      I know this:
      Capital: Athens
      Language: Greek
      Food: Souvlaki and Feta Cheese
      Currency: Euro
      """,
  },    
]

In [19]:
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "What do you know abount {country}?"),
    ("ai", "{answer}"),
])


example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt= example_prompt,
    examples=examples,
    # suffix 필요 없다.
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert, you give short answers."),
    example_prompt,
    ('human', "What do you know abount {country}?"),
])




In [20]:
chain = final_prompt | chat

In [21]:
chain.invoke({'country': 'Germany'})


      I know this:
      Capital: Berlin
      Language: German
      Food: Bratwurst and Sauerkraut
      Currency: Euro
      

AIMessage(content='\n      I know this:\n      Capital: Berlin\n      Language: German\n      Food: Bratwurst and Sauerkraut\n      Currency: Euro\n      ', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-3.5-turbo-0125'}, id='run--04c550fa-f879-488e-8a44-810308059db0-0')

In [22]:
chain.invoke({'country': 'South Korea'})


      I know this:
      Capital: Seoul
      Language: Korean
      Food: Kimchi and Bibimbap
      Currency: South Korean Won
      

AIMessage(content='\n      I know this:\n      Capital: Seoul\n      Language: Korean\n      Food: Kimchi and Bibimbap\n      Currency: South Korean Won\n      ', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-3.5-turbo-0125'}, id='run--d218b99d-3869-4d83-a385-8d6c8a64f149-0')

# LengthBasedExampleSelector

In [27]:
# v0.3
from langchain_core.example_selectors.length_based import LengthBasedExampleSelector
# https://python.langchain.com/api_reference/core/example_selectors/langchain_core.example_selectors.length_based.LengthBasedExampleSelector.html#langchain_core.example_selectors.length_based.LengthBasedExampleSelector

In [24]:
examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

In [26]:
example_prompt = PromptTemplate.from_template("Human: {country}\nAI: {answer}")

In [28]:
example_selector = LengthBasedExampleSelector(
    examples=examples, 
    example_prompt=example_prompt,
    max_length=10,   # 예제의 양을 얼마나 허용할지 정함.
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    # examples= 대신 example_selector= 지정.
    example_selector=example_selector,  # <- max_length= 값에 따라 예제의 양을 정해줄 수 있다.
    suffix="Human: What do you know about {country}?",
    input_variables=['country']    
)

In [29]:
prompt.format(country = 'Brazil')

'Human: What do you know about Brazil?'

In [ ]:
# ↑ 예제가 하나도 선택이 안되었다?

In [30]:
# 위에서 코드 복붙

examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt = example_prompt,
    max_length=80,  # <-- max_length= 값을 변경!
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=['country']
)

print(prompt.format(country="Brazil"))

Human: France
AI: 
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        

Human: What do you know about Brazil?


In [31]:
example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt = example_prompt,
    max_length=180,  # <-- max_length= 값을 변경!
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=['country']
)

print(prompt.format(country="Brazil"))

Human: France
AI: 
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        

Human: Italy
AI: 
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        

Human: What do you know about Brazil?


## CustomExampleSelector (BaseExampleSelector)

In [32]:
# v0.3
from langchain_core.example_selectors.base import BaseExampleSelector

In [35]:
# BaseExampleSelector 의 구현객체를 만드려면
#  상속 받은뒤 select_examples() 과 add_example() 을 반드시 오버라이딩 해주어야 한다.

class RandomExampleSelector(BaseExampleSelector):

    def __init__(self, examples):
        self.examples = examples

    # select_examples()
    # 입력에 따라 어떠한 샘플을 사용할지 select 함.
    
    # 이번 예제에서는 examples 리스트 에서 random 으로 선택하게 하려 함.
    # ※ 이는 얼마든지 복잡하게 만들어 볼수도 있다.
    def select_examples(self, input_variables):
        from random import choice
        return [choice(self.examples)]

    # 이미 존재하는 example 에 example 을 추가하는 method
    def add_example(self, example):
        self.examples.append(example)



In [103]:
example_selector = RandomExampleSelector(
    examples=examples,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=['country']
)

print(prompt.format(country="Brazil"))

Human: Italy
AI: 
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        

Human: What do you know about Brazil?


# Prompt Template 저장 / 읽기 load_prompt()

In [ ]:
# prompt 를 파일로 다룰 경우
#   'JSON' 파일  혹은 'YAML' 파일 로 만들수 있다

## JSON 파일

In [104]:
with open('prompt.json', 'w') as f:
    f.write("""
      {
        "_type":"prompt",
        "template":"What is the capital of {country}",
        "input_variables":["country"]
      }    
    """)

In [105]:
!dir

 D 드라이브의 볼륨: 새 볼륨
 볼륨 일련 번호: 02B1-19A2

 D:\Lang2505\LangWork\02_ModelIO 디렉터리

2025-07-07  오후 08:22    <DIR>          .
2025-07-07  오후 08:22    <DIR>          ..
2025-07-07  오후 07:06    <DIR>          .ipynb_checkpoints
2025-07-07  오후 08:21            35,772 02 Model IO.ipynb
2025-07-07  오후 08:22               150 prompt.json
               2개 파일              35,922 바이트
               3개 디렉터리  460,478,287,872 바이트 남음


In [106]:
# v0.3
from langchain_core.prompts.loading import load_prompt
# https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.loading.load_prompt.html
# Unified method for loading a prompt from LangChainHub or local fs.

In [107]:
prompt = load_prompt('./prompt.json')

prompt

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='What is the capital of {country}')

In [108]:
prompt.format(country='Germany')

'What is the capital of Germany'

## yaml 파일

In [109]:
with open('prompt.yaml', 'w') as f:
    f.write("""
_type: "prompt"
template: "What is the capital of {country}"
input_variables: ["country"]    
    """)

In [110]:
prompt = load_prompt('./prompt.yaml')
print(prompt)
prompt.format(country = 'Germany')

input_variables=['country'] input_types={} partial_variables={} template='What is the capital of {country}'


'What is the capital of Germany'

# Caching

## set_llm_cache(), InMemoryCache

In [111]:
# v0.3
from langchain_core.globals import set_llm_cache
# https://python.langchain.com/api_reference/core/globals/langchain_core.globals.set_llm_cache.html#set-llm-cache
# Set a new LLM cache, overwriting the previous value, if any.

# v0.3
from langchain_core.caches import InMemoryCache
# https://python.langchain.com/api_reference/core/caches/langchain_core.caches.InMemoryCache.html

# Cache that stores things in 'memory'.
# Initialize with empty cache.

In [112]:
set_llm_cache(InMemoryCache())   # LLM 의 모든 response 가 '메모리' 에 저장된다.

In [113]:
chat = ChatOpenAI(temperature=0.1)

In [114]:
# 동일한 질문을 두번 해볼거다.  시간측정하는 함수를 준비해보자
import time
from datetime import timedelta

def check_laptime(message):
    start_time = time.time()
    response = chat.invoke(message)
    end_time = time.time()
    elapsed_time = end_time - start_time # 경과시간
    print('▶ 경과시간 %s' % (str(timedelta(seconds = elapsed_time))))
    print(f'{len(response.content)} 글자: {response.content}\n')

In [115]:
check_laptime("How do you make italian pasta")

▶ 경과시간 0:00:03.707091
897 글자: To make Italian pasta, you will need the following ingredients:

- 2 cups of all-purpose flour
- 2 large eggs
- Pinch of salt

Here is a step-by-step guide to making Italian pasta:

1. On a clean work surface, pour the flour and create a well in the center.
2. Crack the eggs into the well and add a pinch of salt.
3. Using a fork, gradually mix the eggs into the flour until a dough forms.
4. Knead the dough for about 10 minutes until it is smooth and elastic.
5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.
6. After resting, roll out the dough using a pasta machine or a rolling pin until it is thin.
7. Cut the dough into desired shapes, such as fettuccine or spaghetti.
8. Cook the pasta in a large pot of boiling salted water for 2-3 minutes, or until al dente.
9. Drain the pasta and toss with your favorite sauce or toppings.

Enjoy your homemade Italian pasta!



In [116]:
check_laptime("How do you make italian pasta")

▶ 경과시간 0:00:00
897 글자: To make Italian pasta, you will need the following ingredients:

- 2 cups of all-purpose flour
- 2 large eggs
- Pinch of salt

Here is a step-by-step guide to making Italian pasta:

1. On a clean work surface, pour the flour and create a well in the center.
2. Crack the eggs into the well and add a pinch of salt.
3. Using a fork, gradually mix the eggs into the flour until a dough forms.
4. Knead the dough for about 10 minutes until it is smooth and elastic.
5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.
6. After resting, roll out the dough using a pasta machine or a rolling pin until it is thin.
7. Cut the dough into desired shapes, such as fettuccine or spaghetti.
8. Cook the pasta in a large pot of boiling salted water for 2-3 minutes, or until al dente.
9. Drain the pasta and toss with your favorite sauce or toppings.

Enjoy your homemade Italian pasta!



## set_debug()

In [117]:
# v0.3
from langchain_core.globals import set_debug
# https://python.langchain.com/api_reference/core/globals/langchain_core.globals.set_debug.html#langchain_core.globals.set_debug
# Set a new value for the debug global setting.

In [118]:
set_debug(True)

In [119]:
chat.invoke("How do yo make italian pasta")

[llm/start] [llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: How do yo make italian pasta"
  ]
}
[llm/end] [llm:ChatOpenAI] [4.60s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nHere is a step-by-step guide to making Italian pasta:\n\n1. On a clean work surface, pour the flour and make a well in the center.\n2. Crack the eggs into the well and add the salt.\n3. Using a fork, gradually mix the eggs into the flou

AIMessage(content='To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nHere is a step-by-step guide to making Italian pasta:\n\n1. On a clean work surface, pour the flour and make a well in the center.\n2. Crack the eggs into the well and add the salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it is smooth and elastic. If the dough is too dry, add a little water. If it is too wet, add a little more flour.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, roll out the dough using a pasta machine or a rolling pin until it is thin.\n7. Cut the dough into your desired shape, such as fettuccine or spaghetti.\n8. Cook the pasta in a large pot of boiling salted water for 2-3 minutes, or until al dente.\n9. Drain the pasta and toss it with your favorite sauce or

In [120]:
set_debug(False)

## SQLiteCache

In [121]:
# v0.3
from langchain_community.cache import SQLiteCache
# Cache that uses SQLite as a backend.
# Initialize by creating the engine and all tables.
# https://python.langchain.com/api_reference/community/cache/langchain_community.cache.SQLiteCache.html

In [122]:
set_llm_cache(SQLiteCache('cache.db'))

In [123]:
!dir

 D 드라이브의 볼륨: 새 볼륨
 볼륨 일련 번호: 02B1-19A2

 D:\Lang2505\LangWork\02_ModelIO 디렉터리

2025-07-07  오후 08:40    <DIR>          .
2025-07-07  오후 08:40    <DIR>          ..
2025-07-07  오후 07:06    <DIR>          .ipynb_checkpoints
2025-07-07  오후 08:39            52,820 02 Model IO.ipynb
2025-07-07  오후 08:40            32,768 cache.db
2025-07-07  오후 08:22               150 prompt.json
2025-07-07  오후 08:26               103 prompt.yaml
               4개 파일              85,841 바이트
               3개 디렉터리  460,478,238,720 바이트 남음


In [124]:
chat.invoke("How do yo make italian pasta")

AIMessage(content='To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nHere is a step-by-step guide to making Italian pasta:\n\n1. On a clean work surface, pour the flour and make a well in the center.\n2. Crack the eggs into the well and add the salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough starts to form.\n4. Use your hands to knead the dough until it is smooth and elastic. If the dough is too dry, add a little water. If it is too wet, add a little more flour.\n5. Once the dough is ready, cover it with a damp cloth and let it rest for about 30 minutes.\n6. After resting, divide the dough into smaller portions and roll each portion out into a thin sheet using a pasta machine or a rolling pin.\n7. Cut the pasta sheets into your desired shape, such as fettuccine or spaghetti.\n8. Cook the pasta in a large pot of boiling salted water for about 2-3 min

In [125]:
chat.invoke("How do yo make italian pasta")

AIMessage(content='To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nHere is a step-by-step guide to making Italian pasta:\n\n1. On a clean work surface, pour the flour and make a well in the center.\n2. Crack the eggs into the well and add the salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough starts to form.\n4. Use your hands to knead the dough until it is smooth and elastic. If the dough is too dry, add a little water. If it is too wet, add a little more flour.\n5. Once the dough is ready, cover it with a damp cloth and let it rest for about 30 minutes.\n6. After resting, divide the dough into smaller portions and roll each portion out into a thin sheet using a pasta machine or a rolling pin.\n7. Cut the pasta sheets into your desired shape, such as fettuccine or spaghetti.\n8. Cook the pasta in a large pot of boiling salted water for about 2-3 min

## debug, cache 끄려면..

In [126]:
set_debug(False)
set_llm_cache(None)

# OpenAI 모델 비용 확인

In [127]:
# v0.3
from langchain_community.callbacks.manager import get_openai_callback
# Get the OpenAI callback handler in a context manager.
# which conveniently exposes token and cost information.
# Returns: The OpenAI callback handler.
# https://python.langchain.com/api_reference/community/callbacks/langchain_community.callbacks.manager.get_openai_callback.html#get-openai-callback

In [128]:
with get_openai_callback() as usage:
    chat.invoke("What is the recipe for soju")
    print(usage)

Tokens Used: 181
	Prompt Tokens: 14
		Prompt Tokens Cached: 0
	Completion Tokens: 167
		Reasoning Tokens: 0
Successful Requests: 1
Total Cost (USD): $0.0002575


In [129]:
with get_openai_callback() as usage:
    a = chat.invoke("What is the recipe for soju")
    b = chat.invoke("What is the recipe for bread")
    print("🧑🏼\n", a.content)
    print("🧑🏼\n", b.content)
    print(usage)  # 위 두번의 LLM 호출에 대한 전체 비용이 출력된다

🧑🏼
 Ingredients:
- 1 cup of rice
- 1 cup of water
- 1 tablespoon of nuruk (fermentation starter)
- 1 tablespoon of yeast

Instructions:
1. Rinse the rice under cold water until the water runs clear.
2. In a large pot, combine the rice and water and bring to a boil. Reduce heat to low and simmer for 20 minutes, or until the rice is cooked.
3. Remove the pot from heat and let it cool to room temperature.
4. In a separate bowl, mix the nuruk and yeast with a little bit of warm water to form a paste.
5. Add the paste to the cooled rice and mix well.
6. Cover the pot with a clean cloth and let it ferment in a warm, dark place for 3-4 days.
7. After fermentation, strain the mixture through a cheesecloth to remove any solids.
8. Transfer the liquid to a clean bottle and store in the refrigerator.
9. Serve chilled and enjoy your homemade soju!
🧑🏼
 Ingredients:
- 3 1/2 cups all-purpose flour
- 1 packet active dry yeast
- 1 1/2 cups warm water
- 2 tablespoons sugar
- 1 teaspoon salt
- 2 tablespo

In [130]:
print(usage.total_cost)
print(usage.total_tokens)

0.000735
508


# Model Config 저장 / 불러오기

In [131]:
# v0.3
from langchain_openai.llms.base import OpenAI
# https://python.langchain.com/api_reference/openai/llms/langchain_openai.llms.base.OpenAI.html#langchain_openai.llms.base.OpenAI


In [132]:
llm = OpenAI(
    temperature=0.1,
    max_tokens=450,
    model="gpt-4-turbo",
)

# 모델 (config) 저장
llm.save('model.json')

In [133]:
!type model.json

{
    "model_name": "gpt-4-turbo",
    "temperature": 0.1,
    "top_p": 1,
    "frequency_penalty": 0,
    "presence_penalty": 0,
    "n": 1,
    "seed": null,
    "logprobs": null,
    "max_tokens": 450,
    "_type": "openai"
}


In [134]:
# v0.3
from langchain_community.llms.loading import load_llm
# Load LLM from a file.
# https://python.langchain.com/api_reference/community/llms/langchain_community.llms.loading.load_llm.html#langchain_community.llms.loading.load_llm

In [135]:
llm = load_llm('model.json')

D:\Lang2505\LangWork\.venv\Lib\site-packages\langchain_community\llms\openai.py:255: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain_community.chat_models import ChatOpenAI`
  warnings.warn(
D:\Lang2505\LangWork\.venv\Lib\site-packages\langchain_community\llms\openai.py:1089: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain_community.chat_models import ChatOpenAI`
  warnings.warn(


In [136]:
llm

OpenAIChat(client=APIRemovedInV1Proxy, model_kwargs={'model_name': 'gpt-4-turbo', 'temperature': 0.1, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'seed': None, 'logprobs': None, 'max_tokens': 450}, prefix_messages=[])